## PROJECT-3. Решение комплексной бизнес-задачи - увеличение среднего чека

#### ЗАДАЧИ (ФОРМАЛИЗОВАННЫЕ)

1. Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы. Для этого потребуется подготовить и проанализировать имеющиеся данные.
2. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить  продакт-менеджеру.
3. Проанализировать результаты А/Б-теста, проведённого после внедрения фичи, сделать вывод.

## 1.

### Подготовка данных с помощью SQL

#### Код в Metabase >>>

Используем данный код в 
##### Metabase 
для получения файла, который скачаем в 
##### csv-формате 

01. | with data as (
02. |     select distinct user_id,
03. |     resource_id
04. |         from final.carts as c
05. |             join final.cart_items ci
06. |                 on c.id = ci.cart_id
07. |                     where ci.resource_type = 'Course' and c.state = 'successful'
08. |                     order by 1),
09. | counter as (
10. |     select user_id,
11. |     count(resource_id) as cc
12. |         from data
13. |         group by user_id)
14. | select c.user_id, d.resource_id
15. |     from counter as c
16. |         join data as d
17. |             on c.user_id = d.user_id
18. |                 where cc > 1
19. |                 order by 1, 2

##### >>> file 'query_result1''

### ОБРАБОТКА ДАННЫХ

In [2]:
# Импортируем необходимые библиотеки:
import pandas as pd
import numpy as np
import itertools # Импортируем модуль itertools
from scipy.stats import norm # импортируем модули stats, math для определения пределов погрешности и доверительных интервалов
import math

In [3]:
# Открываем файл, скаченный из Metabase, не сгруппированный по идентификатору пользователя (user_id): 
# - здесь мы видим всех пользователей, купивших более 1-го курса
# - 1-ый столбец содержит идентификатор пользователя; 
# - 2-ой столбец содержит номер купленного им курса

df = pd.read_csv('query_result1.csv')
df

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125
...,...,...
34069,2188926,515
34070,2188926,743
34071,2190141,756
34072,2190141,794


In [4]:
# Переименовываем столбцы:
df = df.rename(columns = {'user_id': 'Users', 'resource_id': 'Courses'})
df.head()

,Users,Courses
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [5]:
# Группируем датафрейм df по каждому пользователю, купившему больше 1-го курса:
df_users = df.groupby('Users')['Courses'].apply(lambda x: list(np.unique(x))).reset_index()
df_users

,Users,Courses
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [6]:
# Применим метод combinations модуля itertools для вывода всех возможных комбинаций - курсов из датафрейма df_users, 
# сгруппированного по каждому пользователю, купившему больше 1-го курса - запишем результат в переменную courses_pairs :

courses_pairs = []
for course in df_users['Courses']:
    for pair in itertools.combinations(list(course), 2):
        courses_pairs.append(pair)

In [7]:
courses_pairs

[(516, 1099),
 (356, 357),
 (356, 1125),
 (357, 1125),
 (553, 1147),
 (361, 1138),
 (356, 357),
 (1125, 1140),
 (551, 745),
 (553, 745),
 (551, 1138),
 (553, 568),
 (514, 517),
 (514, 566),
 (517, 566),
 (363, 511),
 (363, 562),
 (363, 563),
 (511, 562),
 (511, 563),
 (562, 563),
 (568, 745),
 (509, 553),
 (509, 745),
 (553, 745),
 (1125, 1144),
 (509, 568),
 (509, 672),
 (568, 672),
 (516, 552),
 (356, 552),
 (357, 571),
 (568, 745),
 (509, 516),
 (509, 568),
 (516, 568),
 (513, 1141),
 (571, 1125),
 (551, 552),
 (551, 744),
 (551, 862),
 (551, 1138),
 (552, 744),
 (552, 862),
 (552, 1138),
 (744, 862),
 (744, 1138),
 (862, 1138),
 (361, 1138),
 (356, 679),
 (571, 745),
 (571, 1099),
 (745, 1099),
 (509, 516),
 (509, 568),
 (509, 1099),
 (516, 568),
 (516, 1099),
 (568, 1099),
 (517, 750),
 (800, 1125),
 (569, 840),
 (509, 568),
 (745, 1125),
 (509, 514),
 (509, 551),
 (509, 745),
 (514, 551),
 (514, 745),
 (551, 745),
 (571, 765),
 (745, 1187),
 (363, 566),
 (513, 1125),
 (1100, 1103

In [8]:
# Исключаем дубликаты, смотрим количество различных уникальных пар курсов, которые встречаются вместе в покупках пользователей:
len(set(courses_pairs))

3989

#### Определяем самую популярную пару курсов

In [9]:
# - для этого создадим отдельный датафрейм:
df_pairs = pd.DataFrame(courses_pairs, columns = ['Course1', 'Course2'])
df_pairs

,Course1,Course2
0,516,1099
1,356,357
2,356,1125
3,357,1125
4,553,1147
...,...,...
40012,765,912
40013,515,743
40014,756,794
40015,756,1185


In [10]:
# Добавляем столбец 'pairs', где выводим приведённые пары курсов вместе:
df_pairs['pairs'] = df_pairs['Course1'].astype(str) + ',' + df_pairs['Course2'].astype(str)
df_pairs

,Course1,Course2,pairs
0,516,1099,"516,1099"
1,356,357,"356,357"
2,356,1125,"356,1125"
3,357,1125,"357,1125"
4,553,1147,"553,1147"
...,...,...,...
40012,765,912,"765,912"
40013,515,743,"515,743"
40014,756,794,"756,794"
40015,756,1185,"756,1185"


In [11]:
# Сгруппируем датафрейм по уникальным парам курсов, приобретённым каждым клиентом и посчитаем количество таких покупок:
df_grouped = df_pairs.groupby('pairs').count().reset_index()
df_grouped = df_grouped[['pairs', 'Course1']].rename(columns = {'Course1': 'pairs_count'})
# df_grouped['pairs_list'] = df_grouped['pairs'].str.split(',')

# Произведём сортировку в датафрейме по количеству от наиболее покупаемых - популярных пар курсов к менее покупаемым:
df_grouped = df_grouped.sort_values('pairs_count', ascending = False)
df_grouped

,pairs,pairs_count
2066,"551,566",797
1624,"515,551",417
911,"489,551",311
1973,"523,551",304
2462,"566,794",290
...,...,...
2875,"672,743",1
1883,"519,1184",1
2873,"672,741",1
2871,"672,1188",1


Самая популярная пара курсов - это [551, 566] с наибольшим количеством покупок (797), которая расположена в самом верху списка.

In [12]:
# Перепроверка: с помощью метода Counter библиотеки collections, считаем количество повторных покупок уникальных пар курсов в 
# списке курсов courses_pairs, который определили с помощью itertools.combinations - записываем результат в переменную popular:

from collections import Counter
popular = Counter([pair for pair in courses_pairs])
popular

Counter({(516, 1099): 25,
         (356, 357): 100,
         (356, 1125): 44,
         (357, 1125): 52,
         (553, 1147): 16,
         (361, 1138): 40,
         (1125, 1140): 1,
         (551, 745): 138,
         (553, 745): 212,
         (551, 1138): 14,
         (553, 568): 83,
         (514, 517): 10,
         (514, 566): 138,
         (517, 566): 21,
         (363, 511): 99,
         (363, 562): 77,
         (363, 563): 33,
         (511, 562): 55,
         (511, 563): 19,
         (562, 563): 53,
         (568, 745): 102,
         (509, 553): 48,
         (509, 745): 59,
         (1125, 1144): 22,
         (509, 568): 46,
         (509, 672): 5,
         (568, 672): 4,
         (516, 552): 12,
         (356, 552): 7,
         (357, 571): 112,
         (509, 516): 35,
         (516, 568): 54,
         (513, 1141): 34,
         (571, 1125): 122,
         (551, 552): 177,
         (551, 744): 16,
         (551, 862): 8,
         (552, 744): 8,
         (552, 862): 6,
         (55

In [13]:
# Смотрим количество уникальных пар курсов с соответствующим им количеством повторных покупок
len(popular)

3989

In [14]:
# Находим самую популярную пару курсов по количеству повторных покупок с помощью метода items() и функции max():
{k: v for k, v in popular.items() if v == max(popular.values())}

{(551, 566): 797}

In [15]:
# Перебираем словарь popular с количеством повторных покупок уникальных пар курсов с помощью метода items() 
# - сортируем список кортежей пар ключ-значение по первому элементу ключа:

sorted_pairs = {k: v for k, v in sorted(popular.items(), key = lambda item: item[0], reverse = False)}
sorted_pairs 
# sorted_pairs = pd.DataFrame(sorted_pairs.items())
# sorted_pairs.columns = ['sorted_pairs', 'pairs_count']
# sorted_pairs = sorted_pairs.sort_values('pairs_count', ascending = False)
# sorted_pairs

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

### Устанавливаем минимальным порог (90-процентиль) для значений повторных покупок уникальных пар курсов, приобретённых каждым клиентом

In [16]:
# Оставим уникальные пары курсов с наибольшим количеством повторных покупок, а минимальные исключим
# - используем 90-процентиль как минимальный порог, запишем данное условие в переменную "mt" (minimum threshold):
mt = np.percentile(df_grouped['pairs_count'], 90)
display(mt)

22.0

In [17]:
# Формируем новый словарь с условием минимального порога для значений повторных покупок - "mt"

mt_popular_pairs = {k:v for k, v in sorted(popular.items(), key = lambda item: item[0], reverse=False) if v > mt}
display(mt_popular_pairs)

{(356, 357): 100,
 (356, 489): 26,
 (356, 514): 35,
 (356, 523): 24,
 (356, 551): 48,
 (356, 571): 103,
 (356, 765): 35,
 (356, 912): 34,
 (356, 1125): 44,
 (357, 502): 23,
 (357, 514): 28,
 (357, 551): 51,
 (357, 571): 112,
 (357, 765): 38,
 (357, 912): 24,
 (357, 1125): 52,
 (358, 570): 25,
 (361, 502): 32,
 (361, 551): 49,
 (361, 672): 23,
 (361, 1138): 40,
 (363, 511): 99,
 (363, 551): 33,
 (363, 562): 77,
 (363, 563): 33,
 (366, 367): 25,
 (366, 489): 38,
 (366, 490): 28,
 (366, 515): 45,
 (366, 523): 44,
 (366, 551): 63,
 (366, 552): 23,
 (366, 566): 26,
 (367, 515): 38,
 (367, 551): 46,
 (368, 504): 23,
 (368, 523): 23,
 (368, 569): 27,
 (368, 572): 29,
 (368, 840): 23,
 (489, 490): 152,
 (489, 502): 67,
 (489, 504): 28,
 (489, 507): 25,
 (489, 514): 78,
 (489, 515): 286,
 (489, 516): 25,
 (489, 517): 28,
 (489, 519): 61,
 (489, 523): 206,
 (489, 551): 311,
 (489, 552): 89,
 (489, 564): 86,
 (489, 566): 188,
 (489, 570): 42,
 (489, 571): 35,
 (489, 679): 77,
 (489, 741): 37,
 (4

In [18]:
# mt_popular_pairs = pd.DataFrame(mt_popular_pairs.items())
# mt_popular_pairs.columns = ['popular_pairs', 'pairs_count']
# mt_popular_pairs = mt_popular_pairs.sort_values('pairs_count', ascending = False)
# mt_popular_pairs

Данный словарь (mt_popular_pairs) с наибольшим количеством повторных покупок будем использовать на этапе оформления таблицы с рекомендациями к основному курсу.

## 2.

### Оформление таблицы с рекомендациями для продакт-менеджера и отдела маркетинга

На данном этапе составляем таблицу с 3-мя столбцами: 
1. курс, к которому идёт рекомендация
2. Курс для рекомендации № 1 (самый популярный).
3. Курс для рекомендации № 2 (второй по популярности).

Для этого , создадим функцию, которая будет рекомендовать два наиболее популярных курса

In [19]:
# Обратимся к первоначальному датафрейму df: выделим курсы, к которым будут идти рекомендации в переменную - main_course
main_course = df['Courses'].unique()
main_course

array([ 516, 1099,  356,  357, 1125,  553, 1147,  361, 1138, 1140,  551,
        745,  568,  514,  517,  566,  363,  511,  562,  563,  509, 1144,
        672,  552,  571,  513, 1141,  744,  862,  679,  750,  800,  569,
        840,  765, 1187, 1100, 1103,  502,  564,  865,  764, 1139, 1186,
        366,  367,  519,  809,  515,  912,  489,  523,  864, 1101, 1146,
        776,  671,  753,  829,  490, 1102,  803,  659,  909,  794,  518,
        907,  777,  908,  360,  813,  835,  741,  752,  814, 1115, 1116,
       1161,  863,  743,  504,  572,  810, 1124, 1128,  742, 1104,  503,
        664,  507,  570, 1185, 1198,  365,  359,  791, 1156,  362, 1184,
        911,  358, 1160,  757,  508, 1181,  755, 1145, 1188,  756,  866,
        749,  368,  364,  834, 1152,  670, 1199,  836, 1201, 1129, 1182,
        902,  837, 1200,  833,  830], dtype=int64)

In [20]:
# Создадим функцию recommend, которая будет рекомендовать 1-ый и 2-ой по популярности курс к основному курсу:
def recommend(main_course):
    course_list = []
    for i in mt_popular_pairs.keys():
        if i[0] == main_course:
            course_list.append((i, mt_popular_pairs[i]))
    popular_pairs_list = sorted(course_list, key = lambda x: x[1], reverse = True)
    return popular_pairs_list[:2] # ставим ограничение для вывода 2-х рекомендаций

In [21]:
recommend(357)

[((357, 571), 112), ((357, 1125), 52)]

In [22]:
# Определяем уникальное количество курсов:
unique_courses = df.Courses.unique()
len(unique_courses)

126

In [23]:
# Теперь, создадим цикл, который для каждого основного курса извлекает 2-ое значение пары согласно уникальным значениям курсов:

recommended = pd.DataFrame(columns = ['Рекомендация 1', 'Рекомендация 2'])
for i in unique_courses:
    if len(recommend(i)) == 2: # задаём условие вывода функцией 3-х рекомендаций
        recommended.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]] # выводим 1 и 2 рекомендации
    elif len(recommend(i)) == 1: # задаём условие вывода функцией 1-ой рекомендации
        recommended.loc[i] = [recommend(i)[0][0][1], np.nan]
    else:                        # задаём условие вывода при отсутствии рекомендации
        recommended.loc[i] = [np.nan, np.nan]

In [24]:
recommended.head()

,Рекомендация 1,Рекомендация 2
516,745,553
1099,NaN,NaN
356,571,357
357,571,1125
1125,1186,NaN


In [25]:
# Подбираем случайные числа: решаем какую функцию использовать - min(), max() или random.choice():

display(recommended['Рекомендация 1'].min())
display(recommend(356))

display(recommended['Рекомендация 1'].max())
display(recommend(1186))

display(recommended['Рекомендация 2'].min())
display(recommend(357))

display(recommended['Рекомендация 2'].max())
display(recommend(1201))

511

[((356, 571), 103), ((356, 357), 100)]

1186.0

[]

357

[((357, 571), 112), ((357, 1125), 52)]

1161

[]

##### Для курсов с пустыми ячейками (nan) - производим замену: используем функцию min(), которая выдаёт минимальный порядковый номер курса с высоким количеством покупок для 1-ой рекомендации, а также функцию random.choice() - случайный выбор курсов в качестве 2-ой рекомендации.

In [26]:
# Заменим пустые значения в рекомендациях на другие курсы, каждый из которых имеет более 100 покупок: 

recommended = pd.DataFrame(columns = ['Рекомендация 1', 'Рекомендация 2'])
for i in unique_courses:
    if len(recommend(i)) == 2: # задаём условие вывода функцией 3-х рекомендаций
        recommended.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]] # выводим 1 и 2 рекомендации
    elif len(recommend(i)) == 1: # задаём условие вывода функцией 1-ой рекомендации
        recommended.loc[i] = [recommend(i)[0][0][1], np.min(i)]
    else:
        recommended.loc[i] = [np.min(i), np.random.choice(i)]

In [27]:
recommended

,Рекомендация 1,Рекомендация 2
516,745,553
1099,1099,1028
356,571,357
357,571,1125
1125,1186,1125
...,...,...
902,902,460
837,837,463
1200,1200,235
833,833,373


In [31]:
# Выводим основной курс и рекомендации 1, 2 к основному курсу:
# Например, посмотрим ТОП-30:

recommended_courses = recommended.reset_index().rename(columns = {'index': 'Основной курс'}).head(30)
recommended_courses

,Основной курс,Рекомендация 1,Рекомендация 2
0,516,745,553
1,1099,1099,1028
2,356,571,357
3,357,571,1125
4,1125,1186,1125
5,553,745,568
6,1147,1147,56
7,361,551,1138
8,1138,1138,512
9,1140,1140,294


## 3.

### На данном этапе, анализируем результаты А/Б-теста после внедрения фичи с предложением о добавлении в корзину 2-го подходящего курса.

После внедрения предложенной системы тестируем гипотезу с помощью А/Б-теста. Для этого запускаем сплит-тест, где все клиенты случайным образом делятся на А-контрольную и Б-тестовую группы. Тестовой группе показываются рекомендации, а контрольной — нет.

3.1
При следующих условиях теста:
- Средняя конверсия в покупку второго курса, до реализации рекомендаций      -    3,2% 
- Ожидаемая средняя конверсия в покупку 2-го курса, после реализации рекомендаций - 4%
- Уровень достоверности  -  95%
- Статистическая мощность - 80%

С помощью калькулятора достоверности А/Б-тестирования (https://www.evanmiller.org/ab-testing/sample-size.html), определяем минимальный размер выборки в каждом варианте для проведения теста: 7866 > округляем до сотых > 7900 - именно столько человек нужно чтобы получить достоверные результаты при сравнении двух вариантов.

3.2
Теперь, оцениваем результаты 3-х недельного сплит-теста:
- в контрольной группе А: 8732 клиента, оформивших заказ; 293 купили больше одного курса
- в тестовой группе Б:    8847 клиента, оформивших заказ; 347 купили больше одного курса

С помощью калькулятора итогов тестирования (https://abtestguide.com/calc/), определяем, что наблюдаемый коэффициент конверсии для варианта Б - 3,92% на 16,89% выше, чем для варианта A - 3,36%; p-значение = 0.0224. С заданной достоверностью 95% и наблюдаемой мощностью 88.69%, мы можем быть уверены, что результат увеличения конверсии является следствием внесенных нами изменений - добавлением фичи, а не результатом случайности.